In [1]:
import os
import re
import requests
import threading
import time
import urllib3
import urllib.error
import urllib.request
import threading
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import urlretrieve

# Colocar o nome do curso aqui
course = 'deep-learning-reinforcement-learning'

# Função para limpar o nome da pasta
def limpar_nome_pasta(nome):
    return re.sub(r'[\\/*?"<>|:]', '', nome)

# Função para fazer download em outro processo
def download_arquivo(file_url, file_path, tentativas=3):
    def download_interno():
        tentativa = 0
        while tentativa < tentativas:
            try:
                urllib.request.urlretrieve(file_url, file_path)
                break
            except urllib.error.ContentTooShortError as e:
                tentativa += 1
                if tentativa == tentativas:
                    print('Falha no download após várias tentativas')
                    print(f'Arquivo: {file_path}')
                    print(f'Link: {file_url}')
                    print(f'Erro: {e}')
    
    thread = threading.Thread(target=download_interno)
    thread.start()

# Função para aguardar página carregar
def aguardar_pagina_carregar(navegador, tempo_maximo=120):
    WebDriverWait(navegador, tempo_maximo).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
    time.sleep(10)

In [2]:
# Carrega o WebDriver com o navegador (Edge ou Chrome)
navegador = webdriver.Edge()

# Prepara o action
action = ActionChains(navegador)

# Maximiza a janela do navegador
navegador.maximize_window()

# Pasta de downloads
pasta_downloads = os.path.join('..', 'downloads')

# Acessa página de login do Coursera
navegador.get('https://www.coursera.org/?authMode=login')

# Espera pelo login ser realizado
try:
    WebDriverWait(navegador, 120).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.rc-AuthenticatedAccountDropdown"))
    )

except TimeoutException:
    print('Login não realizado')
    os._exit(0)

In [3]:
# Vai para página do curso
navegador.get(f'https://www.coursera.org/learn/{course}/home/welcome')

# Espera pelo pop-up de tradução automática
try:
    WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[aria-modal='true']"))
    )
    popup = navegador.find_element(By.CSS_SELECTOR, "div[aria-modal='true']")
    botao_continuar = popup.find_element(By.CSS_SELECTOR, "button[class*='cds-button-primary']")
    botao_continuar.click()

except TimeoutException:
    pass

# Aguarda a página carregar completamente
aguardar_pagina_carregar(navegador)

# Encontra o menu lateral
sider_menu = navegador.find_element(By.CSS_SELECTOR, '[data-e2e="courseNavigation"]')

# Encontra o instituição
institution = sider_menu.find_element(By.TAG_NAME, 'p').text
print(f'Instituição: {institution}')

# Caminho para a pasta da instituição dentro de "downloads"
caminho_institution = os.path.join(pasta_downloads, limpar_nome_pasta(institution))

# Cria a pasta se ela não existir
if not os.path.exists(caminho_institution):
    os.makedirs(caminho_institution)

# Encontra o título do curso
course_title = sider_menu.find_element(By.TAG_NAME, 'h2').get_attribute('title')
print(f'Curso: {course_title}')

# Caminho para a pasta do curso dentro da instituição
caminho_course = os.path.join(caminho_institution, limpar_nome_pasta(course_title))

# Cria a pasta se ela não existir
if not os.path.exists(caminho_course):
    os.makedirs(caminho_course)

# Seleciona o container específico onde os links estão localizados
accordion_container = sider_menu.find_element(By.TAG_NAME, 'nav')
accordion_container = accordion_container.find_element(By.CSS_SELECTOR, '[data-test="rc-WeekCollectionNavigationItem"]')
accordion_container = accordion_container.find_element(By.TAG_NAME, 'ul')

# Busca todos os elementos <a> dentro do container específico
week_elements = accordion_container.find_elements(By.TAG_NAME, 'a')

# Lista para armazenar os pares (descrição da semana, href)
week_info = []

# Itera sobre os elementos encontrados
for week_element in week_elements:
    week = week_element.get_attribute('aria-label').split('.')[0]
    w_href = week_element.get_attribute('href')
    if w_href:
        week_info.append((week, w_href))
print(week_info)

Instituição: IBM
Curso: Aprendizagem profunda e aprendizagem por reforço
[('Semana 1', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/1'), ('Semana 2', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/2'), ('Semana 3', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/3'), ('Semana 4', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/4'), ('Semana 5', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/5'), ('Semana 6', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/6'), ('Semana 7', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/7'), ('Semana 8', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/8'), ('Semana 9', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/home/week/9')]


In [4]:
# Faz o loop nos módulos
for week, w_href in week_info:
    try:
        # Caminho para a pasta do módulo dentro do curso
        caminho_week = os.path.join(caminho_course, limpar_nome_pasta(week))
        print(f'Módulo: {week}')
    
        # Cria a pasta se ela não existir
        if not os.path.exists(caminho_week):
            os.makedirs(caminho_week)
    
        # Navega até a semana 
        navegador.get(w_href)
        
        # Aguarda a página carregar completamente
        aguardar_pagina_carregar(navegador)

        # Aguarda carregar o container do conteúdo do módulo
        WebDriverWait(navegador, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.rc-ModuleLessons"))
        )
        
        # Seleciona o container específico onde os links estão localizados
        module_section_container = navegador.find_element(By.CSS_SELECTOR, "div.rc-ModuleLessons")

        # Aguarde até que os botões do acordeão estejam visíveis e depois colete-os
        accordion_buttons = WebDriverWait(module_section_container, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".cds-AccordionHeader-button"))
        )

        # Clique em cada botão do acordeão para expandir as seções, se não estiverem já expandidas
        for button in accordion_buttons:
            aria_expanded = button.get_attribute('aria-expanded')
            if aria_expanded == "false":
                button.click()
    
        # Busca todos os elementos <a> dentro do container
        lesson_elements = module_section_container.find_elements(By.TAG_NAME, 'a')
        
        # Lista para armazenar os pares (descrição da aula, href)
        lesson_info = []

        # Itera sobre os elementos encontrados
        for lesson_element in lesson_elements:
            lesson = lesson_element.find_element(By.TAG_NAME, 'p').text
            l_href = lesson_element.get_attribute('href')
            if l_href:
                lesson_info.append((lesson, l_href))
        print(lesson_info)
    
        # Faz o loop nas aulas
        for index, (lesson, l_href) in enumerate(lesson_info, start=1):
            try:
                # Caminho para a pasta da aula dentro do módulo
                caminho_lesson = os.path.join(caminho_week, limpar_nome_pasta(f'{index:02d} - {lesson}'))
                print(f'Aula: {index:02d} - {lesson}')
                
                # Cria a pasta se ela não existir
                if not os.path.exists(caminho_lesson):
                    os.makedirs(caminho_lesson)
                
                # Navega até a aula
                navegador.get(l_href)
                
                # Aguarda a página carregar completamente
                aguardar_pagina_carregar(navegador)

                # Pressiona esc para possíveis popups
                action.send_keys(Keys.ESCAPE).perform()
                
                # Procura pelo elemento main na página
                main_element = navegador.find_element(By.TAG_NAME, 'main')
                
                ############ Arquivos txt ############
                # Extrai todo o texto dentro do elemento main
                main_text = main_element.text
                
                # Escreve o texto no arquivo
                with open(os.path.join(caminho_lesson, limpar_nome_pasta(f'{index:02d} - {lesson}.txt')), 'w', encoding='utf-8') as arquivo:
                    arquivo.write(main_text)
                
                ############ Arquivos anexados ############
                # Procura por todos os elementos com data-extension dentro do main_element
                file_elements = main_element.find_elements(By.XPATH, '//*[@data-extension]')
                
                # Faz o download dos arquivos encontrados
                for element in file_elements:
                    file_url = element.get_attribute('data-url')
                    file_name = element.get_attribute('data-name')
                    file_extetion = element.get_attribute('data-extension')
                    download_arquivo(file_url, os.path.join(caminho_lesson, limpar_nome_pasta(file_name)))
                    print(f'Encontrou arquivo {file_extetion}')

                ############ Arquivos de laboratório ############
                # Verifica se a div com a classe 'rc-UngradedLabLauncher' está presente
                if main_element.find_elements(By.CLASS_NAME, 'rc-UngradedLabLauncher') or main_element.find_elements(By.CSS_SELECTOR, 'button[data-track-component="programming_lab_launch_button"]'):
                
                    # Pega a URL atual e adiciona '/lab' ao final da URL
                    lab_url = navegador.current_url + '/lab'
                
                    # Abre uma nova guia e vai para a URL modificada
                    navegador.execute_script("window.open('');")
                    navegador.switch_to.window(navegador.window_handles[1])
                    navegador.get(lab_url)
                
                    # Aguarda a página carregar completamente
                    aguardar_pagina_carregar(navegador)

                    # Espera até que o iframe com o título "lab" esteja presente na página
                    WebDriverWait(navegador, 60).until(
                        EC.presence_of_element_located((By.XPATH, '//iframe[@title="lab"]'))
                    )
                
                    # Procura pelo iframe com o title 'lab'
                    iframe_element = navegador.find_element(By.XPATH, '//iframe[@title="lab"]')
                    
                    # Pega a URL do atributo src do iframe
                    iframe_url = iframe_element.get_attribute('src')
                    
                    # Realiza o download do jupyter notebook
                    download_arquivo(iframe_url, os.path.join(caminho_lesson, limpar_nome_pasta(iframe_url.split("/")[-1])))
                    print(f'Encontrou arquivo {iframe_url.split(".")[-1]}')

                    # Extrai a parte específica da URL que representa a sessão
                    url_lab = f'https://hub.labs.coursera.org/api/workspaceFileBrowser.v1/{iframe_url.split("//")[1].split(".")[0]}?command=Download&arguments=[]'

                    # Faz o download do lab completo
                    download_arquivo(
                        url_lab,
                        os.path.join(caminho_lesson, limpar_nome_pasta(f'{index:02d} - {lesson}.zip'))
                    )
                    navegador.get(url_lab)
                    
                    # Aguarda a página carregar completamente
                    aguardar_pagina_carregar(navegador)

                    # Pressiona esc para possíveis abas de download
                    action.send_keys(Keys.ESCAPE).perform()
                    
                    # Fecha guia atual e volta para guia anterior
                    navegador.close()
                    navegador.switch_to.window(navegador.window_handles[0])
                
                ############ Arquivo de vídeo ############
                # Navega até a aba Downloads
                downloads_tab = WebDriverWait(main_element, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="tabs-root"] button[data-track-component="focused_lex_lecture_tabs_download"]'))
                )
                downloads_tab.click()
                
                # Seleciona o container específico onde os links estão localizados
                downloads_container = main_element.find_element(By.CSS_SELECTOR, '[id$="-panel-DOWNLOADS"]')
                
                # Busca todos os links dentro do container específico
                content_links = downloads_container.find_elements(By.TAG_NAME, "a")
                
                # Busca todos os links dentro do container específico e armazena os hrefs
                content_hrefs = [c_link.get_attribute('href') for c_link in content_links if c_link.get_attribute('href')]
                
                # Faz o download do vídeo
                for c_href in content_hrefs:
                    # Iniciando varredura nos conteúdos
                    if ".mp4" in c_href and "720p" in c_href:
                        # Faz o download do arquivo
                        download_arquivo(c_href, os.path.join(caminho_lesson, limpar_nome_pasta(f'{index:02d} - {lesson}.mp4')))
                        print('Encontrou vídeo')
                        break

                ############ Arquivo de legenda ############
                # Encontrar todos os elementos <track> no vídeo
                track_elements = main_element.find_elements(By.TAG_NAME, "track")
                
                # Faz o download da legenda em pt-BR
                for track in track_elements:
                    if 'Português (Brasil)' in track.get_attribute('label') and 'vtt' in track.get_attribute('src'):
                        download_arquivo(
                            track.get_attribute('src'),
                            os.path.join(caminho_lesson, limpar_nome_pasta(f'{index:02d} - {lesson}.vtt'))
                        )
                        print('Encontrou legenda')
                        break
                
            except TimeoutException:
                # Se acontecer exceção vai para a próxima aula
                pass
    except TimeoutException:
        # Se acontecer exceção vai para o próximo módulo
        pass

# Encerra o processo
navegador.quit()
print('Fim do processo')

Módulo: Semana 1
[('Introdução ao curso', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/tgTfs/course-introduction'), ('Introdução às redes neurais', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/VVfGh/introduction-to-neural-networks'), ('Noções básicas de neurônios', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/yRieI/basics-of-neurons'), ('Redes neurais com o Sklearn', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/VqDFn/neural-networks-with-sklearn'), ('Neurônio em ação', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/A09cN/neuron-in-action'), ('Redes neurais com SKlearn', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/lecture/XQsxv/neural-networks-with-sklearn'), ('Redes neurais com o Sklearn', 'https://www.coursera.org/learn/deep-learning-reinforcement-learning/ungradedLti/E03Ka/neural-networks-with-sklearn'),

FileNotFoundError: [Errno 2] No such file or directory: '..\\downloads\\IBM\\Aprendizagem profunda e aprendizagem por reforço\\Semana 5\\17 - \\17 - .txt'